In [101]:
import os
import os.path as osp
import sys

import pandas as pd
import numpy as np

sys.path.append('../../scripts')
import featurizer
sys.path.append('../../../../../scripts/preprocessing')
import preprocessing

import warnings

import torch

import lightning.pytorch as pl
from lightning.pytorch.callbacks import Callback
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import torchmetrics as tm

sys.path.append('../../scripts')
import dataset_dataloader as dd
import MLP_binary_classifier as mlp_bc

In [2]:
df_btc = preprocessing.load_df('../../../../../../data/Binance/BTC_USDT.csv')

warnings.filterwarnings('ignore')

path_btc = '../../../../../../data/data_new/raw/Binance/BTC_USDT/BTC_USDT.csv'

df_btc_feat = featurizer.load_and_featurize(path_btc)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1811 [00:00<?, ?it/s]

We make use of 90.44726670347875% of the data


0it [00:00, ?it/s]

In [3]:
train_df = df_btc_feat[:int(0.6*len(df_btc_feat))]
val_df = df_btc_feat[int(0.62*len(df_btc_feat)):int(0.75*len(df_btc_feat))]
test_df = df_btc_feat[int(0.75*len(df_btc_feat)):]

train_loader = dd.get_dloader(train_df,16,64,True)
val_loader =  dd.get_dloader(val_df,1024,64,True)
test_loader = dd.get_dloader(test_df,256,64,True)

In [124]:
model = mlp_bc.MLP_binary_classifier([124,64,16])

early_stop_callback = EarlyStopping(monitor="val/p@r=0.25", min_delta=0.01, patience=5, verbose=False, mode="max")

checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val/average_precision",mode = 'max')


In [125]:
trainer = pl.Trainer(accelerator="gpu",callbacks = [early_stop_callback,checkpoint_callback])
trainer.fit(model,train_loader,val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                       | Params
-----------------------------------------------------------------
0 | mlp               | MLP                        | 9.2 K 
1 | Classifier_head   | Linear                     | 17    
2 | s                 | Sigmoid                    | 0     
3 | average_precision | BinaryAveragePrecision     | 0     
4 | pr_curve          | BinaryPrecisionRecallCurve | 0     
-----------------------------------------------------------------
9.2 K     Trainable params
0         Non-trainable params
9.2 K     Total params
0.037     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [126]:
metrics = pd.read_csv('./lightning_logs/version_21/metrics.csv')

In [127]:
metrics.dropna(subset = ['val/average_precision']).drop(['train/loss_step','train/loss_epoch'],axis = 1)



,epoch,step,val/loss,val/average_precision,val/p@r=0.25,val/p@r=0.5,val/p@r=0.75
1,0,61,0.640065,0.862230,0.823741,0.787931,0.775141
4,1,123,0.557191,0.865223,0.829710,0.811723,0.791234
7,2,185,0.599517,0.850618,0.874046,0.818996,0.790323
10,3,247,0.507226,0.829605,0.838828,0.804577,0.801402
14,4,309,0.516015,0.839786,0.854478,0.821942,0.790323
17,5,371,0.536695,0.850351,0.874046,0.844732,0.795824
20,6,433,0.603712,0.811884,0.851301,0.840074,0.809917
23,7,495,0.629152,0.804422,0.870722,0.849442,0.802339


In [128]:
(test_df.timestamp.apply(lambda x : x.day_of_week) == 4).sum()

59

In [129]:
os.listdir('./lightning_logs/version_21/checkpoints')

['epoch=1-step=124.ckpt']

In [130]:
m = mlp_bc.MLP_binary_classifier([34,64,16])

m1 = m.load_from_checkpoint('./lightning_logs/version_21/checkpoints/epoch=1-step=124.ckpt')


In [131]:
test_df_wk = test_df[test_df.timestamp.apply(lambda x : x.day_of_week) == 4]

test_loader = dd.get_dloader(test_df_wk,256,64,False)

In [132]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

m1 = m1.eval()

average_precision = tm.AveragePrecision(task = 'binary')
pr_curve = tm.PrecisionRecallCurve(task="binary")

all_preds = []
all_targets = []

for batch in test_loader:
    
    [batch[0],batch[1]] = batch[0].to(device),batch[1].to(device)
    m1 = m1.to(device)
    
    preds = m1(batch)
    target = batch[1]
    
    all_preds.append(preds)
    all_targets.append(target)
    
ap = torch.hstack(all_preds).flatten()
at = torch.hstack(all_targets).flatten().type(torch.int)

In [133]:
average_precision(ap,at)

tensor(0.6788, device='cuda:0')

In [141]:
precision,recall,thresh = pr_curve(ap,at)

a0 = precision[recall >= 0.1][-1]
a1 = precision[recall >= 0.25][-1]
a2 = precision[recall >= 0.5][-1]
a3 = precision[recall >= 0.75][-1]

a0,a1,a2,a3

(tensor(0.8333, device='cuda:0'),
 tensor(0.6875, device='cuda:0'),
 tensor(0.6364, device='cuda:0'),
 tensor(0.6739, device='cuda:0'))

In [139]:
test_df_wk['y'].sum()/len(test_df_wk)

0.6949152542372882

In [143]:
precision[recall >= 0.1]

tensor([0.6949, 0.6897, 0.6842, 0.6964, 0.6909, 0.7037, 0.6981, 0.6923, 0.6863,
        0.6800, 0.6735, 0.6875, 0.6809, 0.6739, 0.6667, 0.6591, 0.6512, 0.6429,
        0.6341, 0.6500, 0.6667, 0.6842, 0.6757, 0.6667, 0.6571, 0.6471, 0.6364,
        0.6250, 0.6129, 0.6000, 0.5862, 0.5714, 0.5926, 0.6154, 0.6400, 0.6667,
        0.6522, 0.6364, 0.6190, 0.6000, 0.6316, 0.6667, 0.6471, 0.6875, 0.6667,
        0.6429, 0.6154, 0.5833, 0.5455, 0.6000, 0.6667, 0.7500, 0.7143, 0.8333],
       device='cuda:0')